In [18]:
import pandas as pd
import matplotlib as mpl
import numpy as np
#库作用
from nltk.probability import FreqDist
import time
import json 
import codecs
import jieba

In [19]:
#绘制信息
def plot_words(wordList):
    #频度统计吧
    fDist = FreqDist(wordList)
    #print(fDist.most_common())
    print("单词总数: ",fDist.N())
    print("不同单词数: ",fDist.B())
    fDist.plot(10)

In [28]:
# 读取语料
def read_corpus():
    #问题列表
    qList = []
    # 问题的关键词列表
    qList_kw = []
    #答案列表
    aList = []
    #语料地址
    path = "./data/train_pair.json"
    file = open(path, 'r',encoding='utf-8')
    #全部信息
    info = json.loads(file.read())
    #分别处理
    for t in info:
        qList.append(info[t][0])
        #这个函数的作用，含义就是分割词呗
        qList_kw.append(cut(info[t][0]))
        aList.append(info[t][1])
    return qList_kw, qList, aList

In [21]:
#语句类
from jiebaSegment import Seg
class Sentence(object):
    def __init__(self, sentence, id=0):
        self.id = id
        self.origin_sentence = sentence #原句
        self.cuted_sentence = self.cut()#检索式分词结果
    
    # 分词函数
    def cut(self,):
        #搜索引擎模式
        return cut_for_search(self.origin_sentence)

    # 获取分词后的词列表
    def get_cuted_sentence(self):
        return self.cuted_sentence

    # 获取原句子
    def get_origin_sentence(self):
        return self.origin_sentence

    # 设置该句子得分
    def set_score(self, score):
        self.score = score
    

In [29]:
#分割函数
#检索方式的切割，含有同义表达
def cut_for_search(stopwords,sentence):
    # 搜索引擎模式
    seg_list = jieba.cut_for_search(sentence)
    results = []
    #去掉停用词
    for seg in seg_list:
        if seg in stopwords:
            continue
        results.append(seg)

    return results
#直接分割，不含有同义表达
def cut(stopwords,sentence,):
    #使用jieba分割
    seg_list = jieba.cut(sentence, cut_all=False)
    results = []
    #去掉停用词，构建词库
    for seg in seg_list:
        if seg in stopwords:
            continue
        results.append(seg)

    return results

In [30]:
# tfidf模型
from collections import defaultdict
from gensim import corpora, models, similarities
   # 构建其他复杂模型前需要的简单模型
def simple_model(sentences,min_frequency = 1):
    #获取每个句子的分词
    texts =[sentence.get_cuted_sentence() for sentence in sentences]

    # 删除低频词
    frequency = defaultdict(int)#当key不存在返回默认值
    for text in texts:
        for token in text:
            frequency[token] += 1
        #删除频度不够的词语
    texts = [[token for token in text if frequency[token] > min_frequency] for text in texts]
    #建立词袋/词库 词袋模型：该模型忽略掉文本的语法和语序等要素，将文本仅仅看作是若干个词汇的集合
    #文本特征与词的顺序没有关系，

    dictionary = corpora.Dictionary(texts)
    # 将每一句话高维稀疏词频向量
    corpus_simple = [dictionary.doc2bow(text) for text in texts]
    return texts,dictionary,corpus_simple



def TfidfModel(corpus_simple):
    simple_model()
    # 转换模型
    model = models.TfidfModel(corpus_simple)
    #转换后的语料
    corpus = model[corpus_simple]
    # 我们也可以通过save()和load()方法持久化这个相似度矩阵：
    # 创建相似度矩阵
    index = similarities.MatrixSimilarity(corpus)
    return model,corpus,index

In [31]:
#首先是读取语料,将问题描述和问题合并
kw,questionList,answerlist=read_corpus()

TypeError: cut() missing 1 required positional argument: 'sentence'

In [25]:
#然后是处理语料
#读取停用词
stopwords=set();
stopword_filepath = "./stopwordList/stopword.txt"
file_obj = codecs.open(stopword_filepath, 'r', 'utf-8')
while True:
    #按行读取，感觉可以运行一次就保存好，不需要每次都读
    line = file_obj.readline()
    #移除头尾指定的字符
    line=line.strip('\r\n')
    if not line:
        break
    stopwords.add(line)
file_obj.close()
#分割并去除去除停用词
sentences = []
for i in range(0, len(questionList)):
    sentences.append(Sentence(questionList[i], i))
#转换为词袋模型
texts,dictionary,corpus_simple=simple_model(sentences);
#转换为TIDFF模型
model,corpus,index=TfidfModel(corpus_simple);
#计算相似度矩阵并保存

NameError: name 'questionList' is not defined

In [ ]:
# 求最相似的句子
def similarity_k(self, sentence, k):
    sentence_vec = self.sentence2vec(sentence)

    sims = self.index[sentence_vec]
    sim_k = sorted(enumerate(sims), key=lambda item: item[1], reverse=True)[:k]

    indexs = [i[0] for i in sim_k]
    scores = [i[1] for i in sim_k]
    return indexs, scores


In [ ]:
# 对新输入的句子（比较的句子）进行预处理
def sentence2vec(self, sentence):
    sentence = Sentence(sentence, self.seg)
    vec_bow = self.dictionary.doc2bow(sentence.get_cuted_sentence())
    return self.model[vec_bow]

In [15]:
while True:
#处理新输入的句子
    question = input("请输入问题(q退出): ")
    if question == 'q':
        break
    #分割
    s_new = Sentence(question)
    #转化为词袋向量
    vec_bow_new = dictionary.doc2bow(s_new.get_cuted_sentence())
    #转化为Tfidf向量
    vev_tfidf=model[vec_bow_new];
    #计算前k个最匹配的
    sims = index[vev_tfidf]
    sim_k = sorted(enumerate(sims), key=lambda item: item[1], reverse=True)[:k]
    indexs = [i[0] for i in sim_k]
    scores = [i[1] for i in sim_k]
    #计算前k个相似

    question_k = (indexs,scores)

    #输出
    print("亲，我们给您找到的答案是： {}".format(answerlist[question_k[0][0]]))
    for idx, score in zip(*question_k):
        print("same questions： {},                score： {}".format(questionList[idx], score))

TypeError: cut_for_search() missing 1 required positional argument: 'sentence'

In [13]:
a=((1,2))